In [1]:
import fasttext
import fasttext.util

# Đường dẫn file .bin trên Kaggle
bin_path = '/kaggle/input/fasttext-vietnamese-word-vectors-full/cc.vi.300.bin'

# Load model
model = fasttext.load_model(bin_path)
print("Kích thước ban đầu:", model.get_dimension())  # 300

# Giảm chiều xuống 100
fasttext.util.reduce_model(model, 100)
print("Kích thước sau khi giảm:", model.get_dimension())  # 100

# Lưu ra file .vec trong thư mục working để dễ tải về
output_vec_file = '/kaggle/working/cc.vi.100.vec'

def save_model_to_vec(model, output_path):
    words = model.get_words()
    dim = model.get_dimension()
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(f"{len(words)} {dim}\n")  # header
        for w in words:
            vec = model.get_word_vector(w)
            vec_str = ' '.join(map(str, vec))
            f.write(f"{w} {vec_str}\n")

save_model_to_vec(model, output_vec_file)

print(f"Đã lưu vector giảm chiều tại {output_vec_file}")


Kích thước ban đầu: 300
Kích thước sau khi giảm: 100
Đã lưu vector giảm chiều tại /kaggle/working/cc.vi.100.vec


In [2]:
import numpy as np
from sklearn.decomposition import PCA
import os

def reduce_vec_with_pca(input_path, output_path, target_dim=100, max_words=None):
    # Đọc file vector gốc
    with open(input_path, 'r', encoding='utf-8') as f_in:
        header = f_in.readline()
        total_words, dim = map(int, header.strip().split())
        print(f"Tổng số từ: {total_words}, kích thước vector gốc: {dim}")

        if target_dim > dim:
            raise ValueError(f"target_dim ({target_dim}) phải <= dim gốc ({dim})")

        # Giới hạn số từ nếu máy yếu
        if max_words is not None:
            total_words = min(total_words, max_words)

        words = []
        vectors = []

        for i, line in enumerate(f_in):
            if i >= total_words:
                break
            parts = line.rstrip().split(' ')
            words.append(parts[0])
            vec = np.array(parts[1:], dtype=float)
            vectors.append(vec)

        vectors = np.vstack(vectors)
        print(f"Đã load {vectors.shape[0]} vectors.")

    # PCA giảm chiều
    print("Bắt đầu PCA giảm chiều...")
    pca = PCA(n_components=target_dim)
    reduced_vecs = pca.fit_transform(vectors)
    print(f"Vector sau khi giảm chiều: {reduced_vecs.shape}")

    # Lưu file vector mới
    with open(output_path, 'w', encoding='utf-8') as f_out:
        f_out.write(f"{len(words)} {target_dim}\n")
        for w, vec in zip(words, reduced_vecs):
            vec_str = ' '.join(map(str, vec))
            f_out.write(f"{w} {vec_str}\n")

    print(f"File vector đã được lưu tại: {output_path}")

# Cấu hình
input_file = '/kaggle/input/fasttext-vietnamese-word-vectors-full/cc.vi.300.vec'  # Đường dẫn file vector trên Kaggle dataset
output_file = '/kaggle/working/cc.vi.100.vec'           # Nơi lưu file kết quả

reduce_vec_with_pca(input_file, output_file, target_dim=100, max_words=50000)


Tổng số từ: 2000000, kích thước vector gốc: 300
Đã load 50000 vectors.
Bắt đầu PCA giảm chiều...
Vector sau khi giảm chiều: (50000, 100)
File vector đã được lưu tại: /kaggle/working/cc.vi.100.vec
